# Introduction

The Institute for Operations Research and the Management Sciences (INFORMS) holds its Annual Meeting each fall. Every year, student volunteers get stationed throughout the conference facility/ies each day of the conference to help out with the meeting. For this year’s conference, our consulting group is contracted to assist INFORMS in scheduling 14 students across eight shifts – two shifts per day (Sunday, Monday, Tuesday, Wednesday). 




# Selection of Optimization Software

After researching into available optimization softwares, we have decided to use **Gurobipy**[1] package. It is farily to use. To start off, let's create an empty LP model:

In [2]:
from gurobipy import *
import pandas

# create a LP model
model = Model()
model

<gurobi.Model Continuous instance Unnamed: 0 constrs, 0 vars, Parameter changes: LogFile=gurobi.log>

# Variables

In the end, we want to know how to assign students to each shift. For this purpose, let's set up binary variable $X_{i\sim j}$ to denote whether student $i$ will work at $j$, where $j$ encodes the shifts and the location of the shifts, for 

$$i \in A= \{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14\}$$
$$j \in B = \{1s1, 2s1,3m1,4m1,5t1,6t1,5t2,6t2,5t3,6t3,7w1,8w1,7w2,8w2, 5b, 6b, 7b,8b\}$$

For example, $X_{2\sim 6t2} = 1$ means that student 2 is assigned to volunteer during shift 6 at location T2. Notice we also have $5b, 6b, 7b, 8b \in B$ because we want to deal with the special constraints. In this case, $b$ means the backup location. For example, $X_{2\sim 7b} = 1$ means that student 2 is assigned to be the backup personel at shift 7


Now, let's add it to our model.


In [34]:
# Set Variables
i_set = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
j_set = ['1s1', '2s1','3m1','4m1','5t1','6t1','5t2','6t2',
         '5t3','6t3','7w1','8w1','7w2','8w2', '5b', '6b', '7b', '8b']

x = {}
for i in i_set:
    for j in j_set:
        x[str(i)+'~'+j] = model.addVar(
            vtype=GRB.BINARY, 
            name=str(i) + '~' + j
        )
        
model.update()

A typical variable will look like the following outputs. If you are interested in a complete list of variables, refer to **Variable Lists** in the Appendix section

In [5]:
x['2~1s1']

<gurobi.Var 2~1s1>

# Objective function

Along with this variable, our objective is to minimize the total number of shifts assigned to volunteer. Mathematically, our objective function is 

$$ \sum_{i\in A} \sum_{j\in B} X_{i\sim j} $$

Now, we are ready to set the objective function for the model:

In [6]:
# Set objective
model.setObjective(
    quicksum(x[key] for key in x.keys()),
    GRB.MINIMIZE
)
model

<gurobi.Model MIP instance Unnamed: 0 constrs, 504 vars, Parameter changes: LogFile=gurobi.log>

# Constraints

Now, let's deal with the constraints one by one. For each of the constraint formulation, we will print out one example constraint. But if you are interested in the exhaustive list of constraints, please refer to the Appendix.

## Meet the conference demands

The INFORM annual meeting requires certain amount of volunteers during each shifts. Namely: 

$$
\sum_{i\in A}  X_{i\sim 1s1} \geq 2 \quad
\sum_{i\in A}  X_{i\sim 2s1} \geq 2 $$
$$\sum_{i\in A}  X_{i\sim 3m1} \geq 2 \quad
\sum_{i\in A}  X_{i\sim 4m1} \geq 2 $$
$$\sum_{i\in A}  X_{i\sim 5t1} \geq 2 \quad
\sum_{i\in A}  X_{i\sim 6t1} \geq 2 $$
$$\sum_{i\in A}  X_{i\sim 5t2} \geq 1 \quad
\sum_{i\in A}  X_{i\sim 6t2} \geq 1 $$
$$\sum_{i\in A}  X_{i\sim 5t3} \geq 1 \quad
\sum_{i\in A}  X_{i\sim 6t3} \geq 1 $$
$$\sum_{i\in A}  X_{i\sim 7w1} \geq 2 \quad
\sum_{i\in A}  X_{i\sim 8w1} \geq 2 $$
$$\sum_{i\in A}  X_{i\sim 7w2} \geq 1 \quad
\sum_{i\in A}  X_{i\sim 8w2} \geq 1 $$


Now, let's add the constraints into the model:

In [35]:
# Use c to keep track of all constraints
c = {}
# use c1 to keep track constraints for meeting the conference demands
c1 = {}

# 1. meet the conference demands
conference_demands = [2,2,2,2,2,2,1,1,1,1,2,2,1,1]
c_i = 0  # conference index
conference_shift = j_set[:-4] # without '5b'...
for j in conference_shift:
    c['x_i~' + j] = model.addConstr(
        quicksum(x[str(i)+'~' + j] for i in i_set) >= conference_demands[c_i], 
        name='x_i~' + j + ' >= ' + str(conference_demands[c_i])
    )
    c1['x_i~' + j] = c['x_i~' + j]
    c_i += 1

model.update()
c1

{'x_i~1s1': <gurobi.Constr x_i~1s1 >= 2>,
 'x_i~2s1': <gurobi.Constr x_i~2s1 >= 2>,
 'x_i~3m1': <gurobi.Constr x_i~3m1 >= 2>,
 'x_i~4m1': <gurobi.Constr x_i~4m1 >= 2>,
 'x_i~5t1': <gurobi.Constr x_i~5t1 >= 2>,
 'x_i~5t2': <gurobi.Constr x_i~5t2 >= 1>,
 'x_i~5t3': <gurobi.Constr x_i~5t3 >= 1>,
 'x_i~6t1': <gurobi.Constr x_i~6t1 >= 2>,
 'x_i~6t2': <gurobi.Constr x_i~6t2 >= 1>,
 'x_i~6t3': <gurobi.Constr x_i~6t3 >= 1>,
 'x_i~7w1': <gurobi.Constr x_i~7w1 >= 2>,
 'x_i~7w2': <gurobi.Constr x_i~7w2 >= 1>,
 'x_i~8w1': <gurobi.Constr x_i~8w1 >= 2>,
 'x_i~8w2': <gurobi.Constr x_i~8w2 >= 1>}

## Meet the volunteers availability

We did two preprocessings to the availability matrix. 
* First, the original availability use 0 to denote a volunteer is available and 1 to denote otherwise. Such notation is counter-intuitive and we just reverse the matrix (swap the 0's and 1's). 
* Second, we also add columns for $5b, 6b, 7b, 8b$ for programming convenience, since if a student is available for shift $i$, he/she must be availabe for becoming the "back up" at shift $i$

Now, let's read the processed matrix:

In [9]:
# 2. meet the volunteers availability
availability = pandas.read_excel('Availability.xlsx').transpose()
availability

,1s1,2s1,3m1,4m1,5t1,5t2,5t3,5b,6t1,6t2,6t3,6b,7w1,7w2,7b,8w1,8w2,8b
1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1
2,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,1,1,1
3,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0
5,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1
6,1,1,0,0,1,1,1,1,0,0,0,0,1,1,1,1,1,1
7,0,0,0,1,0,0,0,0,1,1,1,1,0,0,0,1,1,1
8,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
10,0,1,1,1,1,1,1,1,0,0,0,0,1,1,1,0,0,0


In order to satisfy the second constraints, we need to have

$$ X_{i\sim j} \leq A_{i\sim j} \quad \text{for } i\in A, j\in B$$

Now, let's add those constraints to the models

In [38]:
for i in i_set:
    for j in j_set:
        constraint_name = str(i)+'~'+j +" <= A_" + str(i) + '~' + j
        c[constraint_name] = model.addConstr(
            x[str(i)+'~'+j] <= availability.loc[i, j],
            name=str(i)+'~'+j +" <= A_" + str(i) + '~' + j
        )

model.update()
# A typical constraint look like this:
c['10~1s1 <= A_10~1s1']

<gurobi.Constr 10~1s1 <= A_10~1s1>

Some typical constraints will look like the following outputs. If you are interested in a complete list of variables, refer to **Constraints List** in the Appendix section

## Special Constraints

There are some extra constraints to consider:
* Student 2 and 5 will at most serve one shifts
* There must be one extra students assigned to each shift on Tuesday and Wednesday (Shifts 5,6,7,8) as "backup". This backup students will not be assigned to a location.

### Special constraint 1

We simply say 

$$ \sum_{j} X_{2j}  \leq 1  \quad \sum_{j} X_{5j}  \leq 1$$

for $j\in B$. Now let's add them to the model:

In [39]:
c["x_2~j <= 1"] = model.addConstr(
    quicksum(x['2~' + j] for j in j_set) <= 1, 
    name="x_2~j <= 1"
)
c["x_5~j <= 1"] = model.addConstr(
    quicksum(x['5~' + j] for j in j_set) <= 1,
    name="x_5~j <= 1"
)

model.update()
# A typical constraint look like this:
c["x_5~j <= 1"]

<gurobi.Constr x_5~j <= 1>

### Special constraint 2

This is where our extra $5b, 6b,7b,8b$ become very useful, we can simply say

$$\sum_{i} X_{i\sim 5b} \geq 1 \quad \sum_{i} X_{i\sim 6b} \geq 1$$
$$\sum_{i} X_{i\sim 7b} \geq 1 \quad \sum_{i} X_{i\sim 8b} \geq 1$$

Now let's add them to the model

In [40]:
c["x_i~5b >= 1"] = model.addConstr(
    quicksum(x[str(i)+'~5b'] for i in i_set) >= 1,
    name="x_i~5b >= 1"
)
c["x_i~6b >= 1"] = model.addConstr(
    quicksum(x[str(i)+'~6b'] for i in i_set) >= 1,
    name="x_i~6b >= 1"
)
c["x_i~7b >= 1"] = model.addConstr(
    quicksum(x[str(i)+'~7b'] for i in i_set) >= 1,
    name="x_i~7b >= 1"
)
c["x_i~8b >= 1"] = model.addConstr(
    quicksum(x[str(i)+'~8b'] for i in i_set) >= 1,
    name="x_i~8b >= 1"
)

model.update()
# A typical constraint look like this:
c["x_i~8b >= 1"]

<gurobi.Constr x_i~8b >= 1>

## Implicit constraints

### No volunteers can be assigned to different location during the same shift

Notice that volunteers can only be assigned to different location during shift 5,6,7,8. So we need to prevent such absurd situation. Thus we should have

$$ X_{i \sim 5t1} + X_{i \sim 5t2} + X_{i \sim 5t3} + X_{i \sim 5b} $$
$$ X_{i \sim 6t1} + X_{i \sim 6t2} + X_{i \sim 6t3} + X_{i \sim 6b} $$
$$ X_{i \sim 7w1} + X_{i \sim 7w2} + X_{i \sim 7b} $$
$$ X_{i \sim 8w1} + X_{i \sim 8w2} + X_{i \sim 8b} $$

for $i\in A$. Now let's add them to the model:

In [41]:
# 4. n
for i in i_set:
    all_location = ['5t1', '5t2', '5t3', '5b']
    c["x_" + str(i) + "~5(k) <= 1"] = model.addConstr(
        quicksum(x[str(i)+'~' + k] for k in all_location) <= 1,
        name="x_" + str(i) + "~5(k) <= 1"
    )
    all_location = ['6t1', '6t2', '6t3', '6b']
    c["x_" + str(i) + "~6(k) <= 1"] = model.addConstr(
        quicksum(x[str(i)+'~' + k] for k in all_location) <= 1,
        name="x_" + str(i) + "~6(k) <= 1"
    )
    all_location = ['7w1', '7w2', '7b']
    c["x_" + str(i) + "~7(k) <= 1"] = model.addConstr(
        quicksum(x[str(i)+'~' + k] for k in all_location) <= 1,
        name="x_" + str(i) + "~7(k) <= 1"
    )
    all_location = ['8w1', '8w2', '8b']
    c["x_" + str(i) + "~8(k) <= 1"] = model.addConstr(
        quicksum(x[str(i)+'~' + k] for k in all_location) <= 1,
        name="x_" + str(i) + "~8(k) <= 1"
    )

model.update()
# A typical constraint look like this:
c["x_1~8(k) <= 1"]

<gurobi.Constr x_1~8(k) <= 1>

# Solution

Now, let's get the results

In [33]:
model.optimize()
print()
print("*****Thus the objective value is " + str(model.ObjVal))

Optimize a model with 2246 rows, 504 columns and 3504 nonzeros
Variable types: 0 continuous, 504 integer (504 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]

Loaded MIP start with objective 26

Presolve removed 2198 rows and 373 columns
Presolve time: 0.00s
Presolved: 48 rows, 131 columns, 262 nonzeros
Variable types: 0 continuous, 131 integer (131 binary)

Root relaxation: cutoff, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        26.00000   26.00000  0.00%     -    0s

Explored 0 nodes (18 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 26 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.600000000000e+01, best bound 2.600000000000e+01, 

# Appendix 

## Variables List

In [18]:
x

{'10~1s1': <gurobi.Var 10~1s1 (value 0.0)>,
 '10~2s1': <gurobi.Var 10~2s1 (value 0.0)>,
 '10~3m1': <gurobi.Var 10~3m1 (value 0.0)>,
 '10~4m1': <gurobi.Var 10~4m1 (value 0.0)>,
 '10~5b': <gurobi.Var 10~5b (value 0.0)>,
 '10~5t1': <gurobi.Var 10~5t1 (value 0.0)>,
 '10~5t2': <gurobi.Var 10~5t2 (value 0.0)>,
 '10~5t3': <gurobi.Var 10~5t3 (value 0.0)>,
 '10~6b': <gurobi.Var 10~6b (value 0.0)>,
 '10~6t1': <gurobi.Var 10~6t1 (value 0.0)>,
 '10~6t2': <gurobi.Var 10~6t2 (value 0.0)>,
 '10~6t3': <gurobi.Var 10~6t3 (value 0.0)>,
 '10~7b': <gurobi.Var 10~7b (value 0.0)>,
 '10~7w1': <gurobi.Var 10~7w1 (value 0.0)>,
 '10~7w2': <gurobi.Var 10~7w2 (value 0.0)>,
 '10~8b': <gurobi.Var 10~8b (value 0.0)>,
 '10~8w1': <gurobi.Var 10~8w1 (value 0.0)>,
 '10~8w2': <gurobi.Var 10~8w2 (value 0.0)>,
 '11~1s1': <gurobi.Var 11~1s1 (value 0.0)>,
 '11~2s1': <gurobi.Var 11~2s1 (value 0.0)>,
 '11~3m1': <gurobi.Var 11~3m1 (value 0.0)>,
 '11~4m1': <gurobi.Var 11~4m1 (value 1.0)>,
 '11~5b': <gurobi.Var 11~5b (value 0.0)>

## Constraints list

In [17]:
c

{'10~1s1 <= A_10~1s1': <gurobi.Constr 10~1s1 <= A_10~1s1>,
 '10~2s1 <= A_10~2s1': <gurobi.Constr 10~2s1 <= A_10~2s1>,
 '10~3m1 <= A_10~3m1': <gurobi.Constr 10~3m1 <= A_10~3m1>,
 '10~4m1 <= A_10~4m1': <gurobi.Constr 10~4m1 <= A_10~4m1>,
 '10~5b <= A_10~5b': <gurobi.Constr 10~5b <= A_10~5b>,
 '10~5t1 <= A_10~5t1': <gurobi.Constr 10~5t1 <= A_10~5t1>,
 '10~5t2 <= A_10~5t2': <gurobi.Constr 10~5t2 <= A_10~5t2>,
 '10~5t3 <= A_10~5t3': <gurobi.Constr 10~5t3 <= A_10~5t3>,
 '10~6b <= A_10~6b': <gurobi.Constr 10~6b <= A_10~6b>,
 '10~6t1 <= A_10~6t1': <gurobi.Constr 10~6t1 <= A_10~6t1>,
 '10~6t2 <= A_10~6t2': <gurobi.Constr 10~6t2 <= A_10~6t2>,
 '10~6t3 <= A_10~6t3': <gurobi.Constr 10~6t3 <= A_10~6t3>,
 '10~7b <= A_10~7b': <gurobi.Constr 10~7b <= A_10~7b>,
 '10~7w1 <= A_10~7w1': <gurobi.Constr 10~7w1 <= A_10~7w1>,
 '10~7w2 <= A_10~7w2': <gurobi.Constr 10~7w2 <= A_10~7w2>,
 '10~8b <= A_10~8b': <gurobi.Constr 10~8b <= A_10~8b>,
 '10~8w1 <= A_10~8w1': <gurobi.Constr 10~8w1 <= A_10~8w1>,
 '10~8w2 <= A

# Reference

[1] http://www.gurobi.com/downloads/get-anaconda